In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Rawap\\Desktop\\App\\datascienceproject\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\Rawap\\Desktop\\App\\datascienceproject'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionconfig:
    root_dir:Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [6]:
class ConfiguartionManager:
    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)-> DataIngestionconfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionconfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [7]:
import os
import urllib.request as request
from src.datascience import logger
import zipfile

In [10]:
## component-Data ingestion
import requests
class DataIngestion:
    def __init__(self, config):
        self.config = config

    # Method: Downloading the zip file
    def download_file(self):
        os.makedirs(os.path.dirname(self.config.local_data_file), exist_ok=True)

        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            print(f"{filename} downloaded! with following info: \n{headers}")
        else:
            print("File already exists")

    # Method: Extract the zip file
    def extract_zip_file(self):
        os.makedirs(self.config.unzip_dir, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(self.config.unzip_dir)
        print(f"Extracted files to {self.config.unzip_dir}")





In [11]:
try:
    config = ConfiguartionManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)

    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-09-20 11:08:07,691: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-20 11:08:07,694: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-20 11:08:07,695: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-09-20 11:08:07,698: INFO: common: created directory at: artifacts]
[2025-09-20 11:08:07,700: INFO: common: created directory at: artifacts/data_ingestion]
artifacts/data_ingestion/data.zip downloaded! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: DFDB:335C6:5E1321:7162E8:68CE6DFB
Accept-Ranges: bytes
Date: Sat, 20 Sep 2025 09:08:08 GMT
Via: 1.1 varnish
X-S